In [1]:
from bs4 import BeautifulSoup as bs
import urllib2,cookielib
import unicodecsv as csv

In [2]:
domain="https://www.practo.com"
state="/delhi/"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [ ]:
#function to open and parse the link-
def soup_up(link):
    req = urllib2.Request(link,headers=hdr)
    print link
    #print req
    html = urllib2.urlopen(req)
    soup = bs(html.read(), "lxml")
    return soup
def scrape_doc(link):
    soup=soup_up(link)
    dic={'name':[],'img':[],'spec':[],'locality':[],'add':[], 'timings':[], 'price':[]}
    dic['name'].append(soup.find(itemprop="name").get_text())
    for x in soup.find_all('h2',class_='doctor-specialties'):
        for y in x.find_all('a'):
            dic['spec'].append(y.get_text())
    dic['img'].append(soup.find('img',class_='doc_avatar')['data-originalsrc'])
    for i in soup.find_all('div', class_='clinic-block listing-row'):
        dic['locality'].append(i.find('h2', class_='strong black').get_text())
        dic['add'].append(i.find(itemprop="streetAddress").get_text())
        try:
            dic['timings'].append(i.find(class_='clinic-timings').get_text().replace("\t", "").replace("\n", ""))
        except(AttributeError):
            dic['timings'].append(None)
        try:
            dic['price'].append(i.find(itemprop="priceRange").get_text().replace("\t", "").replace("\n", ""))
        except(AttributeError):
            dic['price'].append("Undisclosed")
    return dic
soup=soup_up(str(domain)+str(state))
ul = soup.find('ul',class_="oval-blocks-list")
links=[]
for a in ul.find_all('a', href=True):
    for i in range(1,51):
        links.append(str(domain)+str(a['href'])+"?page="+str(i))

https://www.practo.com/delhi/


In [ ]:
#soup=soup_up(links[50])
import pandas as pd
#df = pd.DataFrame()
ax=pd.DataFrame()
for k in links[250:]:
    try:
        soup=soup_up(k)
    except:
        print "Major link error at "+ str(k)
    sub_links=[]
    for a in soup.find_all('a', class_='link doc-name smokeliftDoctorLink fm-target'):
        sub_links.append(a['href'])
    for o in sub_links:
        try:

            ax=ax.append(scrape_doc(o),ignore_index=True)
            
            print len(df)
        except:
            print "error at- " + str(o)

https://www.practo.com/delhi/dermatologist?page=1
https://www.practo.com/delhi/doctor/dr-gaurav-gupta-6-dermatologist-cosmetologist
error at- https://www.practo.com/delhi/doctor/dr-gaurav-gupta-6-dermatologist-cosmetologist
https://www.practo.com/delhi/doctor/dr-r-p-gupta-dermatologist-cosmetologist-1
error at- https://www.practo.com/delhi/doctor/dr-r-p-gupta-dermatologist-cosmetologist-1
https://www.practo.com/delhi/doctor/dr-jyoti-sharma-dermatologist-cosmetologist
error at- https://www.practo.com/delhi/doctor/dr-jyoti-sharma-dermatologist-cosmetologist
https://www.practo.com/delhi/doctor/dr-gaurav-garg-dermatologist-cosmetologist
error at- https://www.practo.com/delhi/doctor/dr-gaurav-garg-dermatologist-cosmetologist
https://www.practo.com/delhi/doctor/dr-richa-sharma-cosmetic-plastic-surgeon
error at- https://www.practo.com/delhi/doctor/dr-richa-sharma-cosmetic-plastic-surgeon
https://www.practo.com/delhi/doctor/dr-sooneita-wagh-markan-dermatologist-cosmetologist-dermatologist-cosm

In [ ]:
links[250]

In [ ]:
ex=pd.read_csv('records-250.csv')

In [ ]:
ew = pd.ExcelWriter('test.xlsx',options={'encoding':'utf-8'})
ex.to_excel(ew)
ew.save()

In [ ]:
scrape_doc(sub_links[4])